# Selenium

Recommended source : https://www.youtube.com/watch?v=Dq8roj-EDPw

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import pickle
import os

import Config_scraper as cs

import time

import pandas as pd
from datetime import datetime



In [9]:
driver=""
wait=""
Espera_boton_mas_comment=1
def iniciar_Chrome():
    ruta="D:\\Git Repositories\\Test\\chromedriver.exe"
    options=Options()
    options.add_argument("--window-size=970,1080")
    options.add_argument("--disable-web-security")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-notifications")
    options.add_argument("--ignore-certificate-error")
    options.add_argument("--no-sandbos")
    options.add_argument("--log-level-3")
    options.add_argument("--allow-running-insecure-content")
    options.add_argument("--no-default-browser-check")
    options.add_argument("--no-first-run")
    options.add_argument("--no-proxy-server")
    options.add_argument("--disable-blink-features=AutomationControlled")

    exp_opt=[
        "enable-automation",
        "ignore-certificate-errors",
        "enable-logging"]
    
    options.add_experimental_option("excludeSwitches",exp_opt)

    prefs = {
        "profile.default_content_setting_values.notifications":2,
        "credentials_enable_service":False
    }

    options.add_experimental_option("prefs",prefs)

    # s=Service(ruta)

    driver = webdriver.Chrome(ruta,options=options)
    driver.set_window_position(0,0)

    return driver
 

def login_instagram():

    if os.path.isfile("instagram.cookies"):
        cookies = pickle.load(open("instagram.cookies","rb"))

        driver.get("https://www.instagram.com/robots.txt")

        for cookie in cookies:
            driver.add_cookie(cookie)
             

        print("login en instagram por cookies")
        driver.get("https://www.instagram.com/")

    else:   
        
        print("login en instagram por credenciales")
        driver.get("https://www.instagram.com/")
        
        #getting the username box
        element=wait.until(ec.visibility_of_element_located((By.NAME,"username")))
        element.send_keys(cs.user)

        element=wait.until(ec.visibility_of_element_located((By.NAME,"password")))
        element.send_keys(cs.password)

        element = driver.find_element(By.CSS_SELECTOR,'button[type="submit"]')
        element.click()

        element=wait.until(ec.element_to_be_clickable((By.XPATH,'//button[text()="Ahora no"]')))
        element.click()


def detect_post_type():
# i assume you're already in the post 

    wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,"div._aatk._aatl")))

    try:
        wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,"div._aatk._aatl")))
        element=driver.find_element(By.CSS_SELECTOR,'div > div > div._aagw')
        type="img"
    except:
        element=driver.find_element(By.CSS_SELECTOR,'article > div > div._aatk._aatl > div > div > div > div > div > div > div > video')
        type="video"

    return type



def get_comments(max_scroll=10):
    collected=[]

    for pages in range(max_scroll):
        comment_boxs = driver.find_elements(By.CSS_SELECTOR,'div > li > div > div > div._a9zr')

        print("comment per page: {}".format(len(comment_boxs)-1))
        if len(comment_boxs)>1: # where 1 is just the post text, >1 is post text and comments
            
            # find the load more comments button and capture data, if it doesn´t exist, load the last comments and break                                
            try:
                WebDriverWait(driver,Espera_boton_mas_comment).until(ec.element_to_be_clickable((By.CSS_SELECTOR,'ul > li > div > button > div > svg')))

                names=driver.find_elements(By.CSS_SELECTOR,'h3 > div > span > a')
                comments=driver.find_elements(By.CSS_SELECTOR,'ul._a9ym>div > li > div > div > div._a9zr > div._a9zs > span')
                
                for n,c in zip(names,comments):
                
                    a=[n.text,c.text]

                    if not(a in collected):
                        collected.append(a)
                    else:                                  
                        break

                next = driver.find_element(By.CSS_SELECTOR,'ul > li > div > button > div > svg')
                next.click()
            except:

                names=driver.find_elements(By.CSS_SELECTOR,'h3 > div > div > div > a')

                comments=driver.find_elements(By.CSS_SELECTOR,"ul._a9ym>div > li > div > div > div._a9zr > div._a9zs > span")
                
                
                for n,c in zip(names,comments):
                
                    a=[n.text,c.text]

                    if not(a in collected):
                        collected.append(a)
                    else:                                  
                        break
                break

        else:              
            print("omitted")
            break

    return collected



def getting_info(max_deep_comments):
    type_post=detect_post_type()

    if type_post=="img":
        
        type="image"
        #locating image
        try:
            element=WebDriverWait(driver,3).until(ec.visibility_of_element_located((By.CSS_SELECTOR,'div[class="_aagu _aato"]>div[class="_aagv"]>img')))
        except:
            element=wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,'div[class="_aagu _aa20 _aato"]>div[class="_aagv"]>img')))
        time.sleep(1)
        #element=wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,'div > div > div._aagw')))
        #url of post
        url=driver.current_url
        #description in instagram
        inner_description=element.get_attribute("alt")
        #video url
        video_url=""
        #image_url source
        image_url=element.get_attribute("src")
        #post text
        try:
            post_text=driver.find_element(By.CSS_SELECTOR,'div > li > div > div > div._a9zr > div._a9zs > h1').text
        except:
            post_text=""
        #reactions (likes)
        reactions =driver.find_element(By.CSS_SELECTOR,"article > div > div._ae65 > div > div > div._ae2s._ae3v._ae3w > section._ae5m._ae5n._ae5o > div > div > div > a > div > span").text
        reactions = reactions+" Me_gusta"
        #post date
        date=driver.find_element(By.CSS_SELECTOR,'div > div > a > div > time').get_attribute("datetime")

    elif type_post=="video":
        
        type="video"
        #locating video
        element=wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,'article > div > div._aatk._aatl > div > div > div > div > div > div > div > video')))
        #url of post
        url=driver.current_url
        #description in instagram
        inner_description=""
        #video url
        video_url=element.get_attribute("src")
        #image_url source (poster in preview)
        image_url=element.get_attribute("poster")
        #post text
        try:
            post_text=driver.find_element(By.CSS_SELECTOR,'div > li > div > div > div._a9zr > div._a9zs > h1').text
        except:
            post_text=""
        #reactions (reproductions)
        try:
            reactions =driver.find_element(By.CSS_SELECTOR,"article > div > div._ae65 > div > div > div._ae2s._ae3v._ae3w > section._ae5m._ae5n._ae5o > div > span > div > span").text
            reactions = reactions+" Reproducciones"
        except:
            reactions =driver.find_element(By.CSS_SELECTOR,"article > div > div._ae65 > div > div > div._ae2s._ae3v._ae3w > section._ae5m._ae5n._ae5o > div > div > div > a > div > span").text
            reactions = reactions+" Me_gusta"
        #post date
        date=driver.find_element(By.CSS_SELECTOR,'div > div > a > div > time').get_attribute("datetime")



    else:
        type="ERROR"
        #url of post
        url=driver.current_url
        #description in instagram
        inner_description="ERROR"
        #video url
        video_url="ERROR"
        #image_url source (poster in preview)
        image_url="ERROR"
        #post text
        post_text="ERROR"
        #reactions (reproductions)
        reactions ="ERROR"
        #post date
        date="ERROR"

    
    extracted_data=[type,url,inner_description,video_url,image_url,post_text,reactions,date,get_comments(max_deep_comments)]

    return extracted_data





def minning(objectives: list,max_posts=25,max_deep_comments=10):


    data=pd.DataFrame([])

    for obj in objectives:

        print("minning: {}".format(obj))
        driver.get("https://www.instagram.com/{}/".format(obj))
        
        element=wait.until(ec.visibility_of_element_located((By.XPATH,"//h2")))
 

        # post por imagen
        elements=driver.find_elements(By.CSS_SELECTOR,'div[class="_aagw"]')

        #click on the first element
        elements[0].click()

        #auxiliar list to store the scraped content
        a=[]

        for i in range(max_posts):

            try:
                wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,"div._aatk._aatl")))

                a.append(getting_info(max_deep_comments))
                
                next_button = wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR,"div > div > div._aaqg._aaqh > button")))

                next_button.click()
            except:
                print("fin de posts")
                break


        df = pd.DataFrame(a, columns = ["p_type",
                "p_url",
                "p_inner_descrption",
                "p_video_url",
                "p_image_url",
                "p_post_text",
                "p_reactions",
                "p_date",
                "p_comments"])

        #cleaning data frame
        df["p_date"] = pd.to_datetime(df["p_date"])
        df["target"]=obj
        #df[['reactions','reactions_type']] = df['p_reactions'].str.split(' ',expand=True)
        #df["reactions"].str.replace(',', '')
        # df["reactions"]=pd.to_numeric(df["reactions"])

        data = data.append(df, ignore_index=True)


    return data




# Run

In [10]:
driver = iniciar_Chrome()

start = datetime.now().strftime("%Y-%m-%d, %H:%M:%S")
print("process started at {}".format(start))

#setting max wait time
wait =WebDriverWait(driver,15)
res = login_instagram()

#saving cookies
cookies = driver.get_cookies()
pickle.dump(cookies,open("instagram.cookies","wb"))

target =["redmedicavitaloficial","grupoemi","emermedica_oficial","medplus_medicina_prepagada","ameasistenciamedica","grupo_coomeva"]
    
    


result=minning(
    target,
    max_posts=350,
    max_deep_comments=15)


result["p_reactions"]=result["p_reactions"].str.replace(',', '')
result[['reactions','reactions_type']] = result['p_reactions'].str.split(' ',expand=True)

# creating two tables to work with
general = result[
    ['p_url',
    'target',
    'p_type',
    'p_image_url',
    'p_video_url',
    'p_post_text',
    'reactions_type',
    'reactions',
    'p_inner_descrption', 
    'p_date']]

general["p_post_text"]=[[x] for x in general["p_post_text"]]

general["p_inner_descrption"]=[[x] for x in general["p_inner_descrption"]]


# use p_url and target as key to join to other table
comments = result[
    ['p_url',
    'target',
    'p_comments']]

comments=comments.explode('p_comments').reset_index(drop=True).dropna()
comments[['comment_author','comment']] = pd.DataFrame(comments.p_comments.tolist(), index= comments.index)

comments.pop("p_comments")

comments["comment"]=[[x] for x in comments["comment"]]
    


absolute_path = os.path.abspath(os.getcwd())

print("saving collected data")
general.to_csv(absolute_path + "\\Output\\general.csv", encoding="utf-8-sig",sep="|")
comments.to_csv(absolute_path + "\\Output\\comments.csv", encoding="utf-8-sig",sep="|")

driver.close()
driver.quit()

finish = datetime.now().strftime("%Y-%m-%d, %H:%M:%S")
print("process finished at {}".format(finish))




process started at 2023-02-25, 08:26:45
login en instagram por cookies
minning: redmedicavitaloficial
comment per page: 1
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
fin de posts
minning: grupoemi
comment per page: 0
omitted
comment per page: 1
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 1
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 1
comment per page: 1
comment per page: 1
comment per page: 0
omitted
comment per page: 1
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted
comment per page: 0
omitted

C:\Users\andre\AppData\Local\Temp\ipykernel_22048\3386813135.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general["p_post_text"]=[[x] for x in general["p_post_text"]]
C:\Users\andre\AppData\Local\Temp\ipykernel_22048\3386813135.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general["p_inner_descrption"]=[[x] for x in general["p_inner_descrption"]]


process finished at 2023-02-25, 08:30:47
